<a href="https://colab.research.google.com/github/aidot-kr/AISecurity/blob/master/6_TopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic Modeling
- 참고DataSet: 네이버 영화리뷰, 해당 폴더에서 ratings.txt 다운로드

1) 다운로드된 파일읽기

In [38]:
# ratings.txt가 있는 파일경로 구성  // Local PC 환경에서의 개발에 적용
#import os
#os.getcwd()
#str_path = '/temp'
#os.chdir(str_path)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/AISecurity

/content/drive/My Drive/AISecurity


In [3]:
ls

2.TFIDF.ipynb           5.OneHotEncode.ipynb       7.ngram.ipynb
3.SimpleCrawling.ipynb  5.Word2Vec-FastText.ipynb  ratings.txt
4_MeCab.ipynb           6.Similarity.ipynb         Sample.ipynb
4_Pre_Processing.ipynb  6.TopicModeling.ipynb      spam.csv


In [4]:
# Colab 서버스펙확인, GPU 할당을 위해서는 메뉴 > 런타임 > 런타임유형변경에서 GPU 선택
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5284498670908342251, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15400733537229074150
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17057333170779152453
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15695549568
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15424137426899416007
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [6]:
import pandas as pd

# ratings.txt는 탭구분자를 기준으로 읽기 실행
doc = pd.read_csv('ratings.txt',sep = '\t' ) 

In [7]:
# ratings.txt를 file open Read도 가능
f = open('ratings.txt', 'r',  encoding='utf-8')
data = f.read()
f.close()

In [8]:
print(len(data))

9248467


In [9]:
doc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [10]:
doc.head(5)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [11]:
doc = doc[doc['document'].notnull()]  # Null값이 있는 필드는 제외
doc = doc[:10000] # 빠른 실습을 위해서 50000개의 Row만 추출

In [12]:
doc.isnull().sum() #null 포함여부 체크

id          0
document    0
label       0
dtype: int64

2) 텍스트데이터 전처리
- 형태소를 사용해서 명사만 추출한다.
- 정확도 개선을 위해서는 불용어, 원형복원, 동철이의어 처리가 필요함

In [13]:
# 한글 불용어 처리를 위한 함수
import re
def clean_sentense(txt):
    pattern = '(\d\d\d-\d\d\d\d-\d\d\d\d)' # 전화번호 제거 (000-0000-0000),\d: 숫자 1개
    txt = re.sub(pattern=pattern, repl='', string=txt)          
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거, a-z 사이의 문자, 
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '<[^>]*>'         # HTML 태그 제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    pattern = '[^\w\s]'         # 특수기호제거
    txt = re.sub(pattern=pattern, repl='', string=txt)
    
    return txt   

In [14]:
!pip3 install konlpy

     |████████████████████████████████| 19.4MB 162kB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 3.8MB 47.6MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [15]:
# 형태소 분석기를 사용하여 영화리뷰 문서 처리 (특정 POS 만추출, 1글자 제외)
from konlpy.tag import Kkma
kkma = Kkma()

# 명사와 동사만 추출하는 함수(미사용)
def extPOS(x):
    allowed_postags=['NNP', 'VV', 'MAG', 'VA']
    texts = kkma.pos(x) 
    texts_out = []
    
    for sent in texts:         
        if sent[1] in allowed_postags:
            texts_out.append(sent[0])
    
    return texts_out


# 문장 추출하는 함수 (한 글자는 제외)
def extWord(x):
    texts = kkma.morphs(x)
    texts_out = []
    for sent in texts:        
        if len(sent) > 1:
            texts_out.append(sent)
    return texts_out

In [16]:
doc['clean_doc2'] = doc['document'].apply(lambda x: clean_sentense(x))   
doc['clean_doc'] = doc['clean_doc2'].apply(lambda x: extWord(x))   

In [17]:
doc.head(20)

,id,document,label,clean_doc2,clean_doc
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1,어릴때보고 지금다시봐도 재밌어요,"[어리, 보고, 지금, 다시, 아도, 재밌, 어요]"
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,"[디자인, 배우, 학생, 으로, 외국, 디자이너, 일구, 전통, 통하, 발전, 문화..."
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음 최고,"[폴리스, 스토리, 시리즈, 부터, 버리, ㄹ게, 하나, 최고]"
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,"[연기, 진짜, 지루, 라고, 생각, 는데, 몰입, 어서, 그리하, 이렇, 진짜, 영화]"
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,"[안개, 자욱, 밤하늘, 초승달, 영화]"
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,"[사랑, 해보, 사람, 라면, 처음, 부터, 까지, 영화]"
6,9279041,완전 감동입니다 다시봐도 감동,1,완전 감동입니다 다시봐도 감동,"[완전, 감동, ㅂ니다, 다시, 아도, 감동]"
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1,개들의 전쟁2 나오나요 나오면 1빠로 보고 싶음,"[전쟁, 나오, 나요, 나오, 빠로]"
8,7477618,굿,1,굿,[]
9,9250537,바보가 아니라 병 쉰 인듯,1,바보가 아니라 병 쉰 인듯,"[바보, 아니]"


3) gensim을 사용한 인코딩 처리
- 토큰화된 문서를 기반으로 각 단어를 정수로 인코딩
- 각 문서내의 단어의 빈도수를 구함
- 예시) dictionary[1]은 정수인코딩 1번으로 처리된 단어를 의미, corpus[1] 1번 문서의 단어 빈도임

In [18]:
from gensim import corpora
dictionary = corpora.Dictionary(doc['clean_doc'])
corpus = [dictionary.doc2bow(text) for text in doc['clean_doc']]
print(dictionary[1], corpus[1])

보고 [(7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1)]


In [19]:
#  문서와 코퍼스의 빈도를 기준 출력
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:100]]

[[('다시', 1), ('보고', 1), ('아도', 1), ('어리', 1), ('어요', 1), ('재밌', 1), ('지금', 1)],
 [('ㅂ니다', 1),
  ('감사', 1),
  ('까지', 1),
  ('나가', 1),
  ('는데', 1),
  ('다는', 1),
  ('디자이너', 1),
  ('디자인', 1),
  ('문화', 1),
  ('발전', 1),
  ('배우', 1),
  ('부럽', 1),
  ('사람', 1),
  ('사실', 1),
  ('산업', 1),
  ('시절', 1),
  ('어렵', 1),
  ('에서', 1),
  ('열정', 1),
  ('외국', 1),
  ('우리나라', 1),
  ('으로', 1),
  ('이루', 1),
  ('일구', 1),
  ('전통', 2),
  ('지키', 1),
  ('통하', 1),
  ('학생', 1)],
 [('ㄹ게', 1),
  ('버리', 1),
  ('부터', 1),
  ('스토리', 1),
  ('시리즈', 1),
  ('최고', 1),
  ('폴리스', 1),
  ('하나', 1)],
 [('는데', 1),
  ('그리하', 1),
  ('라고', 1),
  ('몰입', 1),
  ('생각', 1),
  ('어서', 1),
  ('연기', 1),
  ('영화', 1),
  ('이렇', 1),
  ('지루', 1),
  ('진짜', 2)],
 [('영화', 1), ('밤하늘', 1), ('안개', 1), ('자욱', 1), ('초승달', 1)],
 [('까지', 1),
  ('사람', 1),
  ('부터', 1),
  ('영화', 1),
  ('라면', 1),
  ('사랑', 1),
  ('처음', 1),
  ('해보', 1)],
 [('다시', 1), ('아도', 1), ('ㅂ니다', 1), ('감동', 2), ('완전', 1)],
 [('나오', 2), ('나요', 1), ('빠로', 1), ('전쟁', 1)],
 [],
 [('바보', 1), ('아니', 

In [20]:
len(dictionary)

10139

4) 토픽모델 구축(LDA Model 학습)
- corpus : 문서 벡터
- id2word : 단어ID와 매핑된 단어의 빈도
- num_topics : 가설 토픽 개수
- chunksize : 훈련 알고리즘에 사용되는 문서사이즈, 빠른 학습을 위해서 사이즈 상향 
- Hoffman의 논문에 의하면 Chunksize는 모델 품질에 영향은 일부 있음
- pass : epochs와 같은 용어로 전체 코퍼스에서 모델을 학습시키는 빈도
- iteration : 각각 문서에 대한 반복횟수, passes & iteration 은 많을수록 좋음
- per_word_topics : 각 단어에 대해 가장 가능성이 높은 주제의 내림차순으로 토픽목록 x 피처길이 = 토픽값 계산 

In [21]:
import gensim
from gensim.models.ldamodel import LdaModel

num_topics = 10 # 가설 토픽개수
chunksize = 2000 # 훈련알고리즘에 사용되는 문서의 개수
passes = 20 # 모델 학습빈도
iterations = 400 # 문서 반복횟수
per_word_topics = True # 가능성 높은 주제로 정렬하여 계산

%time lda_model = LdaModel(corpus = corpus, id2word = dictionary,\
                           iterations = iterations, num_topics = num_topics)

CPU times: user 3.9 s, sys: 5.91 ms, total: 3.9 s
Wall time: 3.9 s


In [23]:
# gensim의 멀티코어로 사용 (상위 모델과 동일하나 성능향상), workers는 코어개수를 의미함
# from gensim.models.ldamulticore import LdaMulticore
# from gensim.test.utils import common_corpus, common_dictionary

# %time lda_model = LdaMulticore(corpus, id2word=dictionary, num_topics=5, workers=4)

5) gensim을 CoherenceModel을 사용한 성능평가

In [24]:
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()

# 계산모델의 난이도로 해당 확률 모델의 예측정확도 판정, 수치가 낮을수록 정확성이 높음
print("Perplexity: ", lda_model.log_perplexity(corpus)) 
# 추론된 토픽간의 일관성, 높을수록 의미론적 일관성 높음
print("\nCoherence: ",coherence)

Perplexity:  -8.088956090832205

Coherence:  -4.098857600096797


6) epoch 반복에 따른 coherence 의 변화식별

In [37]:
# import warnings
# import time

# cohrences=[]
# perplexities=[]
# passes=[]
# warnings.filterwarnings('ignore')

# for i in range(10):
    
#     ntopics, nwords = 200, 100
#     if i==0:
#         p=1
#     else:
#         p=i*20
#     tic = time.time()
#     lda4 = LdaMulticore(corpus, id2word=dictionary, num_topics=ntopics, iterations=400, passes=p)
#     print('epoch',p,time.time() - tic)
    
#     cm = CoherenceModel(model=lda4, corpus=corpus, coherence='u_mass')
#     coherence = cm.get_coherence()
#     print("Coherence",coherence)
#     coherences.append(coherence)
#     print('Perplexity: ', lda4.log_perplexity(corpus),'\n\n')
#     perplexities.append(lda4.log_perplexity(corpus))

7) 토픽결과 분석 및 추론

In [27]:
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.022*"영화" + 0.017*"으로" + 0.015*"재밌" + 0.011*"에서" + 0.010*"연기" + 0.009*"주인공" + 0.008*"작품" + 0.008*"남자" + 0.008*"보다" + 0.007*"멋지"')
(1, '0.077*"영화" + 0.017*"ㄴ다" + 0.012*"생각" + 0.012*"최고" + 0.011*"너무" + 0.009*"평점" + 0.008*"지만" + 0.008*"한국" + 0.008*"으로" + 0.008*"아니"')
(2, '0.066*"영화" + 0.024*"네요" + 0.023*"정말" + 0.020*"ㅂ니다" + 0.020*"습니다" + 0.019*"에서" + 0.017*"는데" + 0.012*"좋아하" + 0.012*"평점" + 0.011*"괜찮"')
(3, '0.061*"영화" + 0.019*"감동" + 0.017*"정말" + 0.014*"재미" + 0.012*"시간" + 0.010*"에서" + 0.010*"어요" + 0.008*"재밌" + 0.007*"는데" + 0.007*"까지"')
(4, '0.053*"재밌" + 0.032*"어요" + 0.031*"영화" + 0.027*"는데" + 0.022*"진짜" + 0.019*"너무" + 0.019*"지만" + 0.013*"연기" + 0.013*"어서" + 0.012*"스토리"')
(5, '0.030*"영화" + 0.022*"사랑" + 0.010*"아니" + 0.010*"지만" + 0.010*"배우" + 0.010*"감독" + 0.010*"까지" + 0.010*"가슴" + 0.009*"어서" + 0.007*"사람"')
(6, '0.026*"재미있" + 0.023*"어요" + 0.020*"영화" + 0.020*"완전" + 0.016*"너무" + 0.014*"다시" + 0.013*"아도" + 0.009*"기억" + 0.009*"네요" + 0.009*"대박"')
(7, '0.033*"영화" + 0.014*"장면" + 0.012*"10" + 0.011

In [28]:
for i, topic_list in enumerate(lda_model[corpus]):
    if i==10:
        break
    print('\n',doc['document'][i])
    print(doc['clean_doc'][i])
    print(i,'번째 문서의 topic 비율은',topic_list)


 어릴때보고 지금다시봐도 재밌어요ㅋㅋ
['어리', '보고', '지금', '다시', '아도', '재밌', '어요']
0 번째 문서의 topic 비율은 [(0, 0.012500554), (1, 0.01250132), (2, 0.012502824), (3, 0.012501694), (4, 0.5364981), (5, 0.012500852), (6, 0.012505846), (7, 0.36348382), (8, 0.012502842), (9, 0.012502163)]

 디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.
['디자인', '배우', '학생', '으로', '외국', '디자이너', '일구', '전통', '통하', '발전', '문화', '산업', '부럽', '는데', '사실', '우리나라', '에서', '어렵', '시절', '까지', '열정', '지키', '전통', '사람', '이루', '나가', '다는', '감사', 'ㅂ니다']
1 번째 문서의 topic 비율은 [(2, 0.120916195), (7, 0.8524055)]

 폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.
['폴리스', '스토리', '시리즈', '부터', '버리', 'ㄹ게', '하나', '최고']
2 번째 문서의 topic 비율은 [(0, 0.0111113), (1, 0.011112534), (2, 0.011112139), (3, 0.011113836), (4, 0.7557553), (5, 0.011114351), (6, 0.0111114485), (7, 0.011113696), (8, 0.15534303), (9, 0.011112378)]

 와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지
['연기', '진짜', '지루', '

In [29]:
import pprint
pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.022*"영화" + 0.017*"으로" + 0.015*"재밌" + 0.011*"에서" + 0.010*"연기" + '
  '0.009*"주인공" + 0.008*"작품" + 0.008*"남자" + 0.008*"보다" + 0.007*"멋지"'),
 (1,
  '0.077*"영화" + 0.017*"ㄴ다" + 0.012*"생각" + 0.012*"최고" + 0.011*"너무" + 0.009*"평점" '
  '+ 0.008*"지만" + 0.008*"한국" + 0.008*"으로" + 0.008*"아니"'),
 (2,
  '0.066*"영화" + 0.024*"네요" + 0.023*"정말" + 0.020*"ㅂ니다" + 0.020*"습니다" + '
  '0.019*"에서" + 0.017*"는데" + 0.012*"좋아하" + 0.012*"평점" + 0.011*"괜찮"'),
 (3,
  '0.061*"영화" + 0.019*"감동" + 0.017*"정말" + 0.014*"재미" + 0.012*"시간" + 0.010*"에서" '
  '+ 0.010*"어요" + 0.008*"재밌" + 0.007*"는데" + 0.007*"까지"'),
 (4,
  '0.053*"재밌" + 0.032*"어요" + 0.031*"영화" + 0.027*"는데" + 0.022*"진짜" + 0.019*"너무" '
  '+ 0.019*"지만" + 0.013*"연기" + 0.013*"어서" + 0.012*"스토리"'),
 (5,
  '0.030*"영화" + 0.022*"사랑" + 0.010*"아니" + 0.010*"지만" + 0.010*"배우" + 0.010*"감독" '
  '+ 0.010*"까지" + 0.010*"가슴" + 0.009*"어서" + 0.007*"사람"'),
 (6,
  '0.026*"재미있" + 0.023*"어요" + 0.020*"영화" + 0.020*"완전" + 0.016*"너무" + '
  '0.014*"다시" + 0.013*"아도" + 0.009*"기억" + 0.009*"네요" + 

8) 토픽키워드 시각화
- pyLDAvis 패키지를 사용하며, jupyter notebook에서 잘 동작함

In [31]:
!pip3 install pyLDAvis

     |████████████████████████████████| 1.6MB 6.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=1fd099bb3a1f15bb11dd661849ed73d4dc0a79b43a2b1c749a9c7fd640655625
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [32]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

In [34]:
# 참고용) 문서의 주요 토픽 분석
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=doc['clean_doc'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4.0,0.5362,"재밌, 어요, 영화, 는데, 진짜, 너무, 지만, 연기, 어서, 스토리","[어리, 보고, 지금, 다시, 아도, 재밌, 어요]"
1,1,7.0,0.8523,"영화, 장면, 10, 지만, 지금, 평점, ㄴ다, ㄴ데, 마지막, 아야","[디자인, 배우, 학생, 으로, 외국, 디자이너, 일구, 전통, 통하, 발전, 문화..."
2,2,4.0,0.7559,"재밌, 어요, 영화, 는데, 진짜, 너무, 지만, 연기, 어서, 스토리","[폴리스, 스토리, 시리즈, 부터, 버리, ㄹ게, 하나, 최고]"
3,3,4.0,0.9308,"재밌, 어요, 영화, 는데, 진짜, 너무, 지만, 연기, 어서, 스토리","[연기, 진짜, 지루, 라고, 생각, 는데, 몰입, 어서, 그리하, 이렇, 진짜, 영화]"
4,4,2.0,0.8500,"영화, 네요, 정말, ㅂ니다, 습니다, 에서, 는데, 좋아하, 평점, 괜찮","[안개, 자욱, 밤하늘, 초승달, 영화]"
5,5,5.0,0.5971,"영화, 사랑, 아니, 지만, 배우, 감독, 까지, 가슴, 어서, 사람","[사랑, 해보, 사람, 라면, 처음, 부터, 까지, 영화]"
6,6,6.0,0.5118,"재미있, 어요, 영화, 완전, 너무, 다시, 아도, 기억, 네요, 대박","[완전, 감동, ㅂ니다, 다시, 아도, 감동]"
7,7,0.0,0.8500,"영화, 으로, 재밌, 에서, 연기, 주인공, 작품, 남자, 보다, 멋지","[전쟁, 나오, 나요, 나오, 빠로]"
8,8,0.0,0.1000,"영화, 으로, 재밌, 에서, 연기, 주인공, 작품, 남자, 보다, 멋지",[]
9,9,0.0,0.7000,"영화, 으로, 재밌, 에서, 연기, 주인공, 작품, 남자, 보다, 멋지","[바보, 아니]"


In [35]:
df_dominant_topic.head(1000)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4.0,0.5362,"재밌, 어요, 영화, 는데, 진짜, 너무, 지만, 연기, 어서, 스토리","[어리, 보고, 지금, 다시, 아도, 재밌, 어요]"
1,1,7.0,0.8523,"영화, 장면, 10, 지만, 지금, 평점, ㄴ다, ㄴ데, 마지막, 아야","[디자인, 배우, 학생, 으로, 외국, 디자이너, 일구, 전통, 통하, 발전, 문화..."
2,2,4.0,0.7559,"재밌, 어요, 영화, 는데, 진짜, 너무, 지만, 연기, 어서, 스토리","[폴리스, 스토리, 시리즈, 부터, 버리, ㄹ게, 하나, 최고]"
3,3,4.0,0.9308,"재밌, 어요, 영화, 는데, 진짜, 너무, 지만, 연기, 어서, 스토리","[연기, 진짜, 지루, 라고, 생각, 는데, 몰입, 어서, 그리하, 이렇, 진짜, 영화]"
4,4,2.0,0.8500,"영화, 네요, 정말, ㅂ니다, 습니다, 에서, 는데, 좋아하, 평점, 괜찮","[안개, 자욱, 밤하늘, 초승달, 영화]"
...,...,...,...,...,...
995,995,0.0,0.8200,"영화, 으로, 재밌, 에서, 연기, 주인공, 작품, 남자, 보다, 멋지","[준기, 연기, 돋보이, 재밌]"
996,996,4.0,0.9250,"재밌, 어요, 영화, 는데, 진짜, 너무, 지만, 연기, 어서, 스토리","[평점, 이리, 은지, 공감, 많이, 깨우치, 영화, 10, 그렇게, 어렵, ㄴ지]"
997,997,6.0,0.4620,"재미있, 어요, 영화, 완전, 너무, 다시, 아도, 기억, 네요, 대박","[다시, 아도, 최고]"
998,998,1.0,0.3937,"영화, ㄴ다, 생각, 최고, 너무, 평점, 지만, 한국, 으로, 아니","[진짜, 거치, 없이, 하이, 정말, 최고]"


In [36]:
# 참고용) Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head(100)

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.9735,"영화, 으로, 재밌, 에서, 연기, 주인공, 작품, 남자, 보다, 멋지","[질풍, 노도, 시기, 바람, wind, 처럼, 그저, 흘러가, 하루하루, 재미있,..."
1,1.0,0.9591,"영화, ㄴ다, 생각, 최고, 너무, 평점, 지만, 한국, 으로, 아니","[격투기, 주제, 영화, 대체로, 스토리, 형편없, 는데, 영화, 스토리, 신경, ..."
2,2.0,0.9700,"영화, 네요, 정말, ㅂ니다, 습니다, 에서, 는데, 좋아하, 평점, 괜찮","[엄정, 연기, 안주, 엄정, 연기, 잘하, 뻔히, 당연, 사실, 는데, 영화, 보..."
3,3.0,0.9654,"영화, 감동, 정말, 재미, 시간, 에서, 어요, 재밌, 는데, 까지","[영화, 다만, 엔딩, 에서, 주인공, 배심원, 상대, 늘어놓, 일장, 연설, 너무..."
4,4.0,0.9743,"재밌, 어요, 영화, 는데, 진짜, 너무, 지만, 연기, 어서, 스토리","[내가, 남자, 그렇, ㄴ가, 이거, 긴장감, 흥미진진, 던데, 그렇, ㄴ가, 워낙..."
5,5.0,0.9757,"영화, 사랑, 아니, 지만, 배우, 감독, 까지, 가슴, 어서, 사람","[안개, 에서, 균형, 고는, 자신, 감당, 못하, 탐욕, 부리, 압도적, 운명, ..."
6,6.0,0.9526,"재미있, 어요, 영화, 완전, 너무, 다시, 아도, 기억, 네요, 대박","[독교, 보여주, 유럽, 마녀, 사냥, 다르, 미국, 아주, 그냥, 종교, 전체, ..."
7,7.0,0.9654,"영화, 장면, 10, 지만, 지금, 평점, ㄴ다, ㄴ데, 마지막, 아야","[97, 년도, 영화, 그렇, ㄴ지, 많이, 부족, 장면, 하지만, 97, 년도, ..."
8,8.0,0.9700,"최고, 영화, 드라마, ㅂ니다, 정말, 명작, ㄴ다, 감동적, 아니, 작품","[만약, 시즌, 하면, 성규, 오빠, 주세, 당하, 비장, 머리, 자기, 아니, 라..."
9,9.0,0.9640,"영화, 진짜, 너무, 만들, 는데, 이거, 정말, 네요, ㅂ니다, 에서","[만듬, ost, 부터, 좋앙, 스토리, 구성, 꼼꼼, 하구, 대사, 하나하나, 신..."
